In [1]:
import pandas as pd
import numpy as np
import pathlib as pl
import datetime as dt


p1 = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37')


In [65]:
def import_finometer_txt(folder_path, save_csv=False, resampling=False):

    if len([x for x in folder_path.iterdir() if x.suffix == '.txt']) != 1:
        raise ValueError('''More than one .txt file found in folder, please ensure only one .txt file is present in the folder''')
    else:
        file = [x for x in folder_path.iterdir() if x.suffix == '.txt'][0]

        df = pd.read_csv(
            file,
            sep=';',
            header=0,
            skiprows=8,
            skipfooter=1,
            engine='python',
            infer_datetime_format=True
            )
        df['Time (s)'] = pd.to_datetime(df['Time (s)'], format='%H:%M:%S.%f')
        df['Time (s)'] = df['Time (s)'].dt.strftime('%H:%M:%S.%f')

        if resampling != False:
            resampled = df.groupby(pd.Grouper(key=df['Time (s)'], freq=f'{resampling}')).mean().round(2)
            resampled = resampled.set_index('Time (s)')
            alligned = resampled.groupby(df.index)[df.columns[1:]].max().stack().to_frame().T
        else:
            df = df.set_index('Time (s)')
            alligned = df.groupby(df.index)[df.columns[1:]].max().stack().to_frame().T
            alligned.index = [file.stem]

        if save_csv:
            alligned.to_csv(folder_path / f'imported data for {file.stem}.csv')
            print(f'CSV saved for {file.stem}')

        return alligned

In [66]:
a = import_finometer_txt(p1, resampling='30s')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [60]:
a.iloc[0,1]

0

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

AttributeError: 'Series' object has no attribute 'stack'

In [7]:
def import_finometer_averages(folder_path, time_interval='30s'):
    files = [x for x in folder_path.iterdir() if x.suffix == '.txt']
    dfs = []
    for file in files:
        df = pd.read_csv(file, sep=';', header=0, skiprows=8, skipfooter=1, engine='python', infer_datetime_format=True)
        df['Time (s)'] = pd.to_datetime(df['Time (s)'])
        resampled = df.groupby(pd.Grouper(key='Time (s)', freq=time_interval)).mean().round(2)
        alligned = resampled.groupby(resampled.index)[resampled.columns[1:]].max().stack().to_frame().T
        alligned.index = [file.stem]
        dfs.append(alligned)
    return pd.concat(dfs, axis=0)